## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luderitz,NaN,-26.6481,15.1594,70.27,68,0,18.12,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,48.20,53,20,3.44,few clouds
2,2,Fillmore,US,43.6666,-92.1002,75.20,20,1,23.02,clear sky
3,3,Avarua,CK,-21.2078,-159.7750,82.40,78,75,14.97,light rain
4,4,Saskylakh,RU,71.9167,114.0833,-26.91,100,25,3.40,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.count()

Unnamed: 0             253
City                   253
Country                248
Lat                    253
Lng                    253
Max Temp               253
Humidity               253
Cloudiness             253
Wind Speed             253
Current Description    253
dtype: int64

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

Unnamed: 0             248
City                   248
Country                248
Lat                    248
Lng                    248
Max Temp               248
Humidity               248
Cloudiness             248
Wind Speed             248
Current Description    248
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Fillmore,US,75.20,clear sky,43.6666,-92.1002,
3,Avarua,CK,82.40,light rain,-21.2078,-159.7750,
8,Saint-Pierre,RE,78.80,clear sky,-21.3393,55.4781,
11,Rikitea,PF,76.98,light rain,-23.1203,-134.9692,
12,Maumere,ID,78.26,heavy intensity rain,-8.6199,122.2111,
...,...,...,...,...,...,...,...
693,Sambava,MG,79.14,overcast clouds,-14.2667,50.1667,
695,Miri,MY,77.00,scattered clouds,4.4148,114.0089,
696,Praya,CV,87.80,clear sky,14.9215,-23.5087,
701,Laguna De Perlas,NI,86.00,scattered clouds,12.3429,-83.6712,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location']=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]["name"]
    except:
        print('Hotel not found... skipping')   
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name']!="")]
clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Fillmore,US,75.20,clear sky,43.6666,-92.1002,Country Trails Inn & Suites
3,Avarua,CK,82.40,light rain,-21.2078,-159.7750,Paradise Inn
8,Saint-Pierre,RE,78.80,clear sky,-21.3393,55.4781,Lindsey Hôtel
11,Rikitea,PF,76.98,light rain,-23.1203,-134.9692,Pension Maro'i
12,Maumere,ID,78.26,heavy intensity rain,-8.6199,122.2111,Sylvia Hotel
...,...,...,...,...,...,...,...
693,Sambava,MG,79.14,overcast clouds,-14.2667,50.1667,Melrose
695,Miri,MY,77.00,scattered clouds,4.4148,114.0089,"Mega Hotel, Miri"
696,Praya,CV,87.80,clear sky,14.9215,-23.5087,LT Aparthotel
701,Laguna De Perlas,NI,86.00,scattered clouds,12.3429,-83.6712,Hotel Best View


In [10]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))